# Tubes 2
## Feed Forward Neural Network
___Alvin Sullivan 13515048___

___Albertus Djauhari Djohan 13515054___

___Kevin 13515138___

### Membuat Sebuah Classifier

#### Multi Layer Neural Network
Classifier dibuat dengan sebuah kelas bernama `MultiLayerNN`. Kelas ini berfungsi untuk memodelkan neural network yang mampu melakukan pembelajaran dengan mini-batch stochastic gradient descent. Kelas ini memiliki atribut matriks data input tanpa label, matriks weight dari hidden node, matriks weight dari output node, banyak batch, konstanta learning rate, konstanta tolerance, konstanta momentum, dan banyak epochs. Kelas ini memiliki spesifikasi sebagai berikut.

- Jumlah hidden layer maksimal 10
- Jumlah node dalam setiap hidden layer dapat bervariasi
- Fully-connected layer
- Fungsi aktivasi berupa sigmoid untuk semua hidden layer maupun output layer
- Node output berjumlah 1
- Program memberikan pilihan untuk menggunakan momentum atau tidak
- Program mengimplementasikan mini-batch stochastic gradient descent

Kelas ini memiliki fungsi `train` untuk melakukan pembelajaran mini-batch stochastic gradient descent. Fungsi train melakukan pembelajaran sesuai dengan epochs dan batch yang ditentukan. Untuk setiap batch dalam epochs, dipanggil fungsi `gradient_descent` yang memanggil tiga fungsi lainnya secara berurutan sesuai algoritma gradient descent. Pertama dipanggil fungsi `feed_forward` untuk menentukan output setiap neuron. Kedua dipanggil fungsi `back_propagation` untuk menentukan delta setiap neuron. Ketiga dipanggil fungsi `update_weight` untuk mengubah weight setiap neuron sesuai dengan hasil dari fungsi-fungsi sebelumnya. Setelah seluruh epochs selesai, maka diperoleh sebuah model neural network dengan representasi matriks weight setiap neuron yang sudah diperbarui.

In [13]:
from __future__ import division
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [65]:
class MultiLayerNN:

    def __init__(self, data, hidden_node, output_node,
        num_batch, learning_rate_const, tolerance_const,
        **kwargs):
        
        self.instance = data
        self.w_hidden_node = hidden_node
        self.w_output_node = output_node
        # Gradient Descent Parameters
        self.batch_size = num_batch
        self.learning_rate = learning_rate_const
        self.tolerance = tolerance_const
        self.momentum = kwargs.get('momentum', 0)
        self.epochs = kwargs.get('epochs', 10)
       
    def train(self, instance_target):
        instance_target_t = np.array([instance_target]).T
        batch_iteration = int(np.ceil(self.instance.shape[0] / self.batch_size))
        old_loss = -np.inf
        for step in range(self.epochs):
            loss = 0
            for i in range(batch_iteration):
                start_index = i * self.batch_size
                end_index = i * self.batch_size + self.batch_size
                if end_index > len(instance_target_t):
                    end_index = len(instance_target_t)
                self.gradient_descent(self.instance[start_index:end_index:1], instance_target_t[start_index:end_index:1])
                loss = loss + self.loss_function(self.instance[start_index:end_index:1], instance_target_t[start_index:end_index:1])
        # Print Loss
            print ("Loss after epoch %i: %f" % (step, self.loss_function(self.instance, instance_target)))
            
            if np.abs(loss - old_loss) < self.tolerance:
                break
            old_loss = loss
    
    def feed_forward(self, instance):
        s = list()
        o = list()
        # Feed Forward Hidden Node
        s_temp = instance.dot(self.w_hidden_node[0].T)
        o_temp = self.sigmoid(s_temp)
        s.append(s_temp)
        o.append(o_temp)
        iteration = len(self.w_hidden_node)
        for i in range(1,iteration):
            s_temp = o[i-1].dot(self.w_hidden_node[i].T)
            o_temp = self.sigmoid(s_temp)
            o.append(o_temp)
        # Feed Forward Output Node
        s_out = o[-1].dot(self.w_output_node.T)
        o_out = self.sigmoid(s_out)
        return s, o, s_out, o_out

    def sigmoid(self, X):
        output = 1 / (1 + np.exp(-X))
        return np.matrix(output)

    def loss_function(self, instance, instance_target):
        _,_,_,o_out = self.feed_forward(instance)
        
        squared_error = np.square(instance_target - o_out)
        data_loss = np.sum(squared_error) / 2      
        return data_loss/len(instance)
    
    def back_propagation(self, instance_target, o, o_out):
        d = list()
        # Back Propagation Output Node
        d_temp = np.multiply(np.multiply(o_out, 1-o_out), instance_target-o_out)
        d.insert(0, d_temp)
        # Back Propagation Hidden Node
        d_temp = np.multiply(np.multiply(o[-1], 1-o[-1]), (self.w_output_node.T.dot(d[0].T)).T)
        d.insert(0, d_temp)
        iteration = len(self.w_hidden_node)
        for i in range(iteration-1, 0, -1):
            d_temp = np.multiply(np.multiply(o[i-1], 1-o[i-1]), (self.w_hidden_node[i].T.dot(d[0].T)).T)
            d.insert(0, d_temp)
        return d

    def update_weight(self, instance, o, d):
        # Update Weight Output Node
        self.w_output_node[0] = self.w_output_node[0] + self.momentum * self.w_output_node[0] + self.learning_rate * d[-1].T.dot(o[-1])
        # Update Weight Hidden Node
        iteration = len(self.w_hidden_node)
        for i in range(iteration-1, 0, -1):
            self.w_hidden_node[i] = self.w_hidden_node[i] + self.momentum * self.w_hidden_node[i] + self.learning_rate * d[i].T.dot(o[i-1])
        self.w_hidden_node[0] = self.w_hidden_node[0] + self.momentum * self.w_hidden_node[0] + self.learning_rate * d[0].T.dot(instance)

    def gradient_descent(self, instance, instance_target):
        # Feed Forward
        _,o,_,o_out = self.feed_forward(instance)
        # Back Propagation      
        d = self.back_propagation(instance_target, o, o_out)
        # Update Weight
        self.update_weight(instance, o, d)
    def predict(self, instance):
        _,_,_,o_out = self.feed_forward(instance)
#         return np.argmax(o_out,axis=1)
        return o_out

### Eksekusi Data Weather

Membaca dataset weather dan membentuk data train dan data test

In [5]:
file = "dataset/weather.csv"
data = pd.read_csv(file)
data = data.apply(preprocessing.LabelEncoder().fit_transform)
data = data.values
X = data[:, 1:]
y = data[:, 0]
print('Data train:\n', X)
print('Data test:\n', y)

Data train:
 [[1 0 0 0]
 [1 0 1 0]
 [1 0 0 1]
 [2 0 0 1]
 [0 1 0 1]
 [0 1 1 0]
 [0 1 1 1]
 [2 0 0 0]
 [0 1 0 1]
 [2 1 0 1]
 [2 1 1 1]
 [2 0 1 1]
 [1 1 0 1]
 [2 0 1 0]]
Data test:
 [2 2 0 1 1 1 0 2 2 1 2 0 0 1]


#### Classifier yang Dibuat

Inisialisasi matriks weight hidden node dan output node

In [9]:
np.random.seed(0)
w1 = np.random.randn(4, 4) / np.sqrt(4)
w2 = np.random.randn(3, 4) / np.sqrt(3)
w_output_node = np.random.randn(1, 3)
w_hidden_node = list()
w_hidden_node.append(w1)
w_hidden_node.append(w2)
print('Matriks weight hidden node:\n', w_hidden_node)
print('Matriks weight output node:\n', w_output_node)

Matriks weight hidden node:
 [array([[ 0.88202617,  0.2000786 ,  0.48936899,  1.1204466 ],
       [ 0.933779  , -0.48863894,  0.47504421, -0.0756786 ],
       [-0.05160943,  0.20529925,  0.07202179,  0.72713675],
       [ 0.38051886,  0.06083751,  0.22193162,  0.16683716]]), array([[ 0.86260696, -0.11844818,  0.18074972, -0.4931124 ],
       [-1.47396936,  0.37736687,  0.49908247, -0.42848917],
       [ 1.31044344, -0.83967841,  0.02641869, -0.10807065]])]
Matriks weight output node:
 [[ 1.53277921  1.46935877  0.15494743]]


Membuat model neural network

In [86]:
multiLayerNN = MultiLayerNN(X, w_hidden_node, w_output_node, 1, 1e-3, 1e-6, momentum = 0.001, epochs = 20)
multiLayerNN.train(y)
print('Matriks weight hidden node:\n', multiLayerNN.w_hidden_node)
print('Matriks weight output node:\n', multiLayerNN.w_output_node)

x_test = np.matrix([[2,1,1,1],[1,0,0,1]])
y_test = multiLayerNN.predict(x_test)
print(y_test)

Loss after epoch 0: 4.500000
Loss after epoch 1: 4.500000
Matriks weight hidden node:
 [matrix([[ 15.95221118,   3.62461523,   8.81021135,  20.21791464],
        [ 17.17073925,  -8.91155706,   8.52705499,  -1.77655068],
        [  1.47103551,   3.90126729,   1.85972512,  12.30878436],
        [  5.27095517,   0.96660187,   3.57494671,   3.61036983]]), matrix([[ 20.77470711,   3.6723422 ,   6.40119898,  -4.41970522],
        [-21.60871077,  12.46439502,  11.86689302,  -3.4623616 ],
        [ 24.98278373, -13.56777945,   1.26286241,  -0.78455863]])]
Matriks weight output node:
 [[ 38.95841948  32.87630791  12.66473973]]
[[ 1.]
 [ 1.]]


### Pembagian Tugas
1. Alvin Sullivan - 13515048 - 
2. Albertus Djauhari - 13515054 - 
3. Kevin - 13515138 - 